In [4]:
import numpy as np
from scipy.stats import binom
from scipy.stats import truncnorm
from scipy.stats import gamma
from scipy.special import factorial

In [5]:
# Parameters 

h = 1/7
pho = 2/3
g = 2/3
R = 19
T = 7*R

In [10]:
def K(k, betas):
    K = np.zeros((4,4))
    K[0, 0], K[0, 1], K[1, 1], K[1, 2], K[2, 2], K[2, 3] = np.exp(-betas[k]*h), 1-np.exp(-betas[k]*h), np.exp(-h*pho), 1-np.exp(-h*pho), np.exp(-h*g), 1-np.exp(-h*g)
    return(K)

In [50]:
def division(P, Q):
    Q_new = Q
    for s in np.argwhere(Q == 0):
        if s[0]!=1 or s[1]!=2:
             Q_new[s[0]][s[1]] = 1
    #print(Q_new)
    M = np.divide(P, Q_new)
    for s in np.argwhere(Q == 0):
        M[s[0]][s[1]] = 0
    return M

In [51]:
# Initial settings
X1 = np.array([948, 0, 1, 0])
X2 = np.array([1689, 0, 1, 0])
X3 = np.array([3466, 0, 1, 0])
X4 = np.array([1894, 0, 1, 0])

def PAL(y, theta, R=19):
    X = np.array([X1, X2, X3, X4]) 
    X_all = np.array([X for r in range(R+1)])
    B = theta[:16].reshape(4,4)
    L = np.zeros((R, 4))
    for r in range(R):
        G = np.zeros((7, 4, 4, 4))
        for i in range(6):
            I =  X[:, 2]   
            betas = 1/8000 * np.dot(B, I)
            for k in range(4):
                G[i][k] = np.multiply(np.outer(np.ones(4), X[k]), K(k, betas))
                X[k] = np.transpose(np.dot(np.transpose(np.ones(4)), G[i][k]))
        I =  X[:, 2]   
        betas = 1/8000 * np.dot(B, I)
        for k in range(4):
            G[6][k] = np.multiply(np.outer(np.ones(4), X[k]), K(k, betas))
            M = np.zeros((4,4))
            Q = np.zeros((4,4))
            Q[1][2] = theta[k+16]
            for i in range(7):
                M = M + np.multiply(G[i][k], Q)
            Y_bar = np.zeros((4,4))
            Y_bar[1][2] = y[r][k]
            G_bar = np.multiply(np.outer(np.ones(4), np.ones(4))-Q, G[i][k]) + np.multiply(Y_bar, np.multiply(G[6][k], division(Q, M)))
            X[k] = np.transpose(np.dot(np.transpose(np.ones(4)), G_bar))
            L[r][k] = -np.dot(np.transpose(np.ones(4)), np.dot(M, np.ones(4))) + np.dot(np.transpose(np.ones(4)), np.dot(np.multiply(Y_bar, np.log(M)), np.ones(4))) - np.dot(np.transpose(np.ones(4)), np.dot(np.log(factorial(Y_bar)), np.ones(4)))
    return L

In [52]:
theta = np.concatenate((np.random.gamma(5, 1, 16), truncnorm.rvs(a=0, b=1, loc=0.5, scale=0.5, size=4)), axis=None)
Y = np.array([[0,0,1,1],[0,2,6,1],[0,2,4,2],[23,73,63,11],[63,208,173,41],[73,207,171,27],[66,150,143,7],[26,40,87,29],[17,18,33,12],[3,4,13,6],[2,6,16,5],[1,6,11,3],[0,1,6,5],[0,2,2,2],[0,1,3,0],[0,1,4,6],[0,1,3,0],[2,1,7,1], [1,1,6,2]])
PAL(Y, theta, R=19)

/tmp/ipykernel_1037/341686687.py:7: RuntimeWarning: divide by zero encountered in divide
  M = np.divide(P, Q_new)
/tmp/ipykernel_1037/1823553022.py:33: RuntimeWarning: divide by zero encountered in log
  L[r][k] = -np.dot(np.transpose(np.ones(4)), np.dot(M, np.ones(4))) + np.dot(np.transpose(np.ones(4)), np.dot(np.multiply(Y_bar, np.log(M)), np.ones(4))) - np.dot(np.transpose(np.ones(4)), np.dot(np.log(factorial(Y_bar)), np.ones(4)))
/tmp/ipykernel_1037/1823553022.py:33: RuntimeWarning: invalid value encountered in multiply
  L[r][k] = -np.dot(np.transpose(np.ones(4)), np.dot(M, np.ones(4))) + np.dot(np.transpose(np.ones(4)), np.dot(np.multiply(Y_bar, np.log(M)), np.ones(4))) - np.dot(np.transpose(np.ones(4)), np.dot(np.log(factorial(Y_bar)), np.ones(4)))


array([[ -15.40806819,  -15.41690456,  -16.28689861,  -16.11514292],
       [          nan,          -inf,  -27.19846142,  -16.11514292],
       [          nan,          -inf,  -22.0643394 ,  -17.32982327],
       [         -inf,          -inf, -271.01687879,  -38.8327825 ],
       [         -inf,          -inf,          -inf, -123.18525699],
       [         -inf,          -inf,          -inf,  -45.25462849],
       [         -inf,          -inf, -159.06718786,  -16.99389536],
       [         -inf,          -inf,  -53.27053911,  -40.54263074],
       [         -inf,          -inf,  -17.67574787,  -18.18314695],
       [         -inf,          -inf,  -25.2278189 ,  -17.03568077],
       [         -inf,          -inf,  -22.19668187,  -17.09394128],
       [         -inf,          -inf,  -25.53018874,  -17.69238035],
       [          nan,          -inf,  -30.71759404,  -16.81832958],
       [          nan,          -inf,  -36.79555753,  -17.68425206],
       [          nan,          -i

In [37]:
Q = np.zeros((4, 4))
print(np.argwhere(Q == 0))

[[0 0]
 [0 1]
 [0 2]
 [0 3]
 [1 0]
 [1 1]
 [1 2]
 [1 3]
 [2 0]
 [2 1]
 [2 2]
 [2 3]
 [3 0]
 [3 1]
 [3 2]
 [3 3]]
